In [1]:
%pip install lxml
#%pip install rdflib
import os
import s3fs
import gzip
import xml.etree.ElementTree as ET
from lxml import etree
import pandas as pd

os.environ["AWS_ACCESS_KEY_ID"] = 'GMHBU5SUKVUN1A81Y8UY'
os.environ["AWS_SECRET_ACCESS_KEY"] = 'o+beqI9AemFNrUL4c0OSTxDbOhg4llbcwl0ScqEm'
os.environ["AWS_SESSION_TOKEN"] = 'eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiJHTUhCVTVTVUtWVU4xQTgxWThVWSIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNzY5NjEyMTg2LCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6Imx1Y2FzLmN1bXVuZWxAZW5zYWUuZnIiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiZXhwIjoxNzcwNzk4MzU4LCJmYW1pbHlfbmFtZSI6IkN1bXVuZWwiLCJnaXZlbl9uYW1lIjoiTHVjYXMiLCJncm91cHMiOlsiVVNFUl9PTllYSUEiLCJzdGF0YXBwLXNlZ21lZGljIl0sImlhdCI6MTc3MDE5MzU1NywiaXNzIjoiaHR0cHM6Ly9hdXRoLmxhYi5zc3BjbG91ZC5mci9hdXRoL3JlYWxtcy9zc3BjbG91ZCIsImp0aSI6Im9ucnRydDplNzAyY2NmOS05N2FjLTNkMDgtNDk0YS1iZWQ1M2EzMzYyMzQiLCJuYW1lIjoiTHVjYXMgQ3VtdW5lbCIsInBvbGljeSI6InN0c29ubHkiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJsYWIiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiIsImRlZmF1bHQtcm9sZXMtc3NwY2xvdWQiXX0sInJlc291cmNlX2FjY2VzcyI6eyJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX19LCJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIiwiZGVmYXVsdC1yb2xlcy1zc3BjbG91ZCJdLCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGdyb3VwcyBlbWFpbCIsInNpZCI6Ijc4ODAyYmFkLTFjZTEtODFkOC1mYjI4LTQwNWY4MWQ0ZGVmNSIsInN1YiI6ImUyZDc4NjRjLTcwMzItNDI0ZC04OTA2LWU0ZjhiNDFjYzAwMyIsInR5cCI6IkJlYXJlciJ9.UhD0r4MQ2qUkefOj6PjF-awWui47amNDnE3PLOZnuc30X6GTsfDb4N9Aqm2acDms4ez0btnYSOi6_to_-wQlsw'
os.environ["AWS_DEFAULT_REGION"] = 'us-east-1'
fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},
    key = os.environ["AWS_ACCESS_KEY_ID"], 
    secret = os.environ["AWS_SECRET_ACCESS_KEY"], 
    token = os.environ["AWS_SESSION_TOKEN"])

path = "lab/dblp (1).xml.gz"
dtd_path = "lab/dblp.dtd"
rdf_path="lab/dblp.nt.gz"
"""
# Download the DTD locally
with fs.open(dtd_path, "rb") as f_dtd:
    with open("dblp.dtd", "wb") as f_local:
        f_local.write(f_dtd.read())"""


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 36.6 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


'\n# Download the DTD locally\nwith fs.open(dtd_path, "rb") as f_dtd:\n    with open("dblp.dtd", "wb") as f_local:\n        f_local.write(f_dtd.read())'

In [13]:
authors_by_pid = {}
dois_by_pub = {}

signature_to_pid = {}
pub_to_signatures = {}
pub_to_pids = {}
pid_to_label = {}

RDFS_LABEL = "http://www.w3.org/2000/01/rdf-schema#label"

with fs.open(rdf_path, "rb") as f:
    with gzip.GzipFile(fileobj=f) as gz:
        for i, line in enumerate(gz):

            line = line.decode("utf-8", errors="ignore").strip()
            if not line or line.startswith("#"):
                continue

            parts = line.split(" ", 2)
            if len(parts) != 3:
                continue

            subj, pred, obj = parts
            subj = subj.strip("<>")
            pred = pred.strip("<>")
            obj = obj.strip(" .").strip("<>")

            # ---------- PID -> label ----------
            if pred == RDFS_LABEL:
                pid_to_label[subj] = obj.strip('"')

            # ---------- signature -> PID ----------
            elif pred.endswith("signatureCreator"):
                signature_to_pid[subj] = obj

            # ---------- publication -> signature ----------
            elif pred.endswith("hasSignature") or pred.endswith("authoredBy"):
                pub_to_signatures.setdefault(subj, []).append(obj)

            # ---------- DOI ----------
            elif pred.endswith("doi"):
                dois_by_pub[subj] = obj.strip('"')

            if i % 1_000_000 == 0 and i > 0:
                print(f"Processed {i:,} lines")

# ---------- FINAL MERGE ----------
for sig, pid in signature_to_pid.items():
    label = pid_to_label.get(pid)
    if label:
        authors_by_pid[pid] = label

for pub, sigs in pub_to_signatures.items():
    pids = []

    for sig in sigs:
        pid = signature_to_pid.get(sig)
        if pid:
            pids.append(pid)

    if pids:
        pub_to_pids[pub] = pids

pub_to_pids[pub] = list(set(pids))

print("Authors:", len(authors_by_pid))
print("DOIs:", len(dois_by_pub))


Processed 1,000,000 lines
Processed 2,000,000 lines
Processed 3,000,000 lines
Processed 4,000,000 lines
Processed 5,000,000 lines
Processed 6,000,000 lines
Processed 7,000,000 lines
Processed 8,000,000 lines
Processed 9,000,000 lines
Processed 10,000,000 lines
Processed 11,000,000 lines
Processed 12,000,000 lines
Processed 13,000,000 lines
Processed 14,000,000 lines
Processed 15,000,000 lines
Processed 16,000,000 lines
Processed 17,000,000 lines
Processed 18,000,000 lines
Processed 19,000,000 lines
Processed 20,000,000 lines
Processed 21,000,000 lines
Processed 22,000,000 lines
Processed 23,000,000 lines
Processed 24,000,000 lines
Processed 25,000,000 lines
Processed 26,000,000 lines
Processed 27,000,000 lines
Processed 28,000,000 lines
Processed 29,000,000 lines
Processed 30,000,000 lines
Processed 31,000,000 lines
Processed 32,000,000 lines
Processed 33,000,000 lines
Processed 34,000,000 lines
Processed 35,000,000 lines
Processed 36,000,000 lines
Processed 37,000,000 lines
Processed 

In [24]:
pd.DataFrame(pub_to_pids.items(), columns=["pid", "name"])

,pid,name
0,https://dblp.org/rec/reference/vision/Singh14,[https://dblp.org/pid/78/459-1]
1,https://dblp.org/rec/reference/vision/Wong14,[https://dblp.org/pid/69/220]
2,https://dblp.org/rec/reference/vision/Pont14b,[https://dblp.org/pid/99/2633]
3,https://dblp.org/rec/reference/vision/Alexander14,[https://dblp.org/pid/37/6152]
4,https://dblp.org/rec/reference/vision/Fukui14,[https://dblp.org/pid/01/1485]
...,...,...
8101479,https://dblp.org/rec/persons/Codd71b,[https://dblp.org/pid/c/EFCodd]
8101480,https://dblp.org/rec/persons/Codd71,[https://dblp.org/pid/c/EFCodd]
8101481,https://dblp.org/rec/persons/Hall74,[https://dblp.org/pid/82/5885]
8101482,https://dblp.org/rec/persons/Codd69,[https://dblp.org/pid/c/EFCodd]


In [4]:
with fs.open(f"s3://lab/conf_net.parquet", "wb") as f:
            df.to_parquet(f, index=False)

In [ ]:
articles=[]

with fs.open(path, 'rb') as f:
    with gzip.GzipFile(fileobj=f) as gz:
        context = etree.iterparse(
            gz, events=("end",), tag="article", recover=True
        )

        for event, elem in context:
            title = elem.findtext("title")
            journal = elem.findtext("journal")
            pub_date = elem.findtext("year")
            doi = elem.findtext("ee")

            # Article-level DBLP PID
            article_pid = elem.get("key")

            # Collect authors + IDs
            authors = []

            for a in elem.findall(".//author"):
                name = a.text     # DBLP person ID
                orcid = a.get("orcid")
                if name:
                    authors.append({
                        "name": name,
                        "orcid": orcid
                    })

            articles.append({
                "title": title,
                "journal": journal,
                "year": pub_date,
                "authors": authors,
                "dblp_uri": article_pid,
                "doi": doi
            })

            elem.clear()

df = pd.DataFrame(articles)


In [2]:
articles=[]

with fs.open(path, 'rb') as f:
    with gzip.GzipFile(fileobj=f) as gz:
        context = etree.iterparse(
            gz, events=("end",), tag="inproceedings", recover=True
        )

        for event, elem in context:
            title = elem.findtext("title")
            conf = elem.findtext("booktitle")
            pub_date = elem.findtext("year")
            doi = elem.findtext("ee")

            # Article-level DBLP PID
            article_pid = elem.get("key")

            # Collect authors + IDs
            authors = []

            for a in elem.findall(".//author"):
                name = a.text     # DBLP person ID
                orcid = a.get("orcid")
                if name:
                    authors.append({
                        "name": name,
                        "orcid": orcid
                    })

            articles.append({
                "title": title,
                "conf": conf,
                "year": pub_date,
                "authors": authors,
                "dblp_uri": article_pid,
                "doi": doi
            })

            elem.clear()

df = pd.DataFrame(articles)


In [3]:
df

,title,conf,year,authors,dblp_uri,doi
0,The Future of Classic Data Administration: Obj...,SWEE,1998,"[{'name': 'Arnon Rosenthal', 'orcid': None}]",www/org/mitre/future,http://www.mitre.org/support/swee/rosenthal.html
1,Some Patterns of Convincing Software Engineeri...,Denert Award,2020,"[{'name': 'Lutz Prechelt', 'orcid': None}]",books/sp/22/Prechelt22,https://doi.org/10.1007/978-3-030-83128-8_2
2,Static Worst-Case Analyses and Their Validatio...,Denert Award,2020,"[{'name': 'Peter Wgemann', 'orcid': None}]",books/sp/22/Wagemann22,https://doi.org/10.1007/978-3-030-83128-8_11
3,Crossing Disciplinary Borders to Improve Requi...,Denert Award,2020,"[{'name': 'Anne Hess', 'orcid': None}]",books/sp/22/Hess22,https://doi.org/10.1007/978-3-030-83128-8_7
4,What You See Is What You Get: Practical Effect...,Denert Award,2020,"[{'name': 'Jonathan Immanuel Brachthuser', 'or...",books/sp/22/Brachthauser22,https://doi.org/10.1007/978-3-030-83128-8_3
...,...,...,...,...,...,...
3749511,"Toward a Modern Geography of Minds, Machines, ...",PT-AI,2011,"[{'name': 'Selmer Bringsjord', 'orcid': None},...",series/sapere/BringsjordG13,https://doi.org/10.1007/978-3-642-31674-6_11
3749512,Emotional Control-Conditio Sine Qua Non for Ad...,PT-AI,2011,"[{'name': 'Claudius Gros', 'orcid': '0000-0002...",series/sapere/Gros13,https://doi.org/10.1007/978-3-642-31674-6_14
3749513,Becoming Digital: Reconciling Theories of Digi...,PT-AI,2011,"[{'name': 'Harry Halpin', 'orcid': None}]",series/sapere/Halpin13,https://doi.org/10.1007/978-3-642-31674-6_15
3749514,Generative Artificial Intelligence.,PT-AI,2011,"[{'name': 'Tijn van der Zant', 'orcid': None},...",series/sapere/ZantKS13,https://doi.org/10.1007/978-3-642-31674-6_8


In [9]:

import re
df[df["authors"].apply(lambda x: bool(re.compile(r"^[A-Za-z\s-]+$").search(str(x))))]


,title,conf,year,authors
0,The Future of Classic Data Administration: Obj...,SWEE,1998,Arnon Rosenthal
1,Some Patterns of Convincing Software Engineeri...,Denert Award,2020,Lutz Prechelt
2,Static Worst-Case Analyses and Their Validatio...,Denert Award,2020,Peter Wgemann
3,Crossing Disciplinary Borders to Improve Requi...,Denert Award,2020,Anne Hess
4,What You See Is What You Get: Practical Effect...,Denert Award,2020,Jonathan Immanuel Brachthuser
...,...,...,...,...
3749504,"Machine Intentionality, the Moral Status of Ma...",PT-AI,2011,David Leech Anderson
3749509,Practical Introspection as Inspiration for AI.,PT-AI,2011,Sam Freed
3749510,"""Computational Ontology and Deontology"".",PT-AI,2011,Raffaela Giovagnoli
3749512,Emotional Control-Conditio Sine Qua Non for Ad...,PT-AI,2011,Claudius Gros


In [2]:
from lxml import etree
import gzip

direct_set = set()

with fs.open(path, "rb") as f:
    with gzip.GzipFile(fileobj=f) as gz:
        context = etree.iterparse(gz, events=("end",), tag="inproceedings", recover=True)
        for event, elem in context:
            # collect child tags
            for child in elem:
                if child.tag is not None:
                    direct_set.add(child.tag)

            # clear memory: remove element and its previous siblings
            elem.clear()
            while elem.getprevious() is not None:
                del elem.getparent()[0]

print(sorted(direct_set))

['author', 'booktitle', 'cdrom', 'cite', 'crossref', 'editor', 'ee', 'month', 'note', 'number', 'pages', 'publnr', 'stream', 'title', 'url', 'volume', 'year']


In [30]:
chunks = []
n_chunks = 50
chunk_size = len(df) // n_chunks

for i in range(n_chunks):
    start = i * chunk_size
    end = (i + 1) * chunk_size if i < n_chunks - 1 else len(df)  # last chunk includes remainder
    
    chunk = df.iloc[start:end].copy()
    
    # Split authors column into multiple columns
    #authors_df = chunk['authors'].str.split(';', expand=True)
    #authors_df.columns = [f'author{i+1}' for i in range(authors_df.shape[1])]
    
    # Combine with original chunk
    #chunk = pd.concat([chunk.drop(columns=['authors']), authors_df], axis=1)
    
    chunk.to_parquet(f"inpro/inproceedings_chunk_{i}.parquet", index=False)

NameError: name 'df' is not defined

In [35]:

import pandas as pd
import re
from collections import defaultdict
import glob

ego_data = defaultdict(lambda: {"publications": []}) #set permet de créer des clefs si absentes et de les actualiser si présentes (en définissant la classe des clefs)

chunk_files = glob.glob("inpro/inproceedings_chunk_*.parquet")

for file in chunk_files:
    df_chunk = pd.read_parquet(file)
    for _, row in df_chunk.iterrows():
        authors = row['authors'].split(';')
        #authors = [a for a in authors if not re.compile(r"\d").search(a)]
        for ego in authors:
            coauthors = set(authors) - {ego}
            
            ego_data[ego]["publications"].append({
                "coauthors": list(coauthors),
                "source": row.get("conf"),
                "year": row.get("year"),
                "title": row.get("title"),
                "doi": row.get("doi")
            })

In [36]:
rows = []
for ego, data in ego_data.items():
    for pub in data["publications"]:
        rows.append({
            "ego": ego,
            "coauthors": ",".join(sorted(pub["coauthors"])),
            "source": pub.get("source"),
            "year": pub.get("year"),
            "title": pub.get("title"),
            "doi": pub.get("doi")
        })

inp_df = pd.DataFrame(rows)


In [37]:
inp_df

,ego,coauthors,source,year,title,doi
0,Sanish Rai,Xiaolin Hu 0002,SpringSim (TMS),2018,Hybrid agent-based and graph-based modeling fo...,http://dl.acm.org/citation.cfm?id=3213189
1,Sanish Rai,Xiaolin Hu 0002,IAT,2013,Behavior Pattern Detection for Data Assimilati...,https://doi.org/10.1109/WI-IAT.2013.106
2,Sanish Rai,"Minghao Wang, Xiaolin Hu 0002",SpringSim (ADS),2015,A graph-based agent-oriented model for buildin...,http://dl.acm.org/citation.cfm?id=2872548
3,Sanish Rai,Xiaolin Hu 0002,WSC,2017,Data assimilation with sensor-informed resampl...,https://doi.org/10.1109/WSC.2017.8247862
4,Sanish Rai,Xiaolin Hu 0002,COMPSAC,2016,Graph Based Agent Oriented Model for Tunnel Si...,https://doi.org/10.1109/COMPSAC.2016.223
...,...,...,...,...,...,...
12855429,Juan Gutierrez,"Ashley Houston-King, Cliff Freeman, Eli Tucke...",FabLearn/Constructionism,2023,Youth Pedagogical Development in Youth Teachin...,https://doi.org/10.1145/3615430.3615444
12855430,Ashley Houston-King,"Cliff Freeman, Eli Tucker-Raymond, Katherine ...",FabLearn/Constructionism,2023,Youth Pedagogical Development in Youth Teachin...,https://doi.org/10.1145/3615430.3615444
12855431,Cliff Freeman,"Ashley Houston-King, Eli Tucker-Raymond, Kath...",FabLearn/Constructionism,2023,Youth Pedagogical Development in Youth Teachin...,https://doi.org/10.1145/3615430.3615444
12855432,Katherine K. Frankel,"Ashley Houston-King, Cliff Freeman, Eli Tucke...",FabLearn/Constructionism,2023,Youth Pedagogical Development in Youth Teachin...,https://doi.org/10.1145/3615430.3615444


In [38]:
art_df["Type"]="art"
inp_df["Type"]="inp"
ego_df=pd.concat([art_df,inp_df], axis=0, ignore_index=True)
ego_df

,ego,coauthors,source,year,title,doi,Type
0,Kodai Shimosato,Norimichi Ukita,CoRR,2024,Inpainting-Driven Mask Optimization for Object...,https://doi.org/10.48550/arXiv.2403.15849,art
1,Kodai Shimosato,Norimichi Ukita,IEEE Access,2021,Multi-Modal Data Fusion for Land-Subsidence Im...,https://doi.org/10.1109/ACCESS.2021.3120133,art
2,Norimichi Ukita,Kodai Shimosato,CoRR,2024,Inpainting-Driven Mask Optimization for Object...,https://doi.org/10.48550/arXiv.2403.15849,art
3,Norimichi Ukita,"Arnau Raventos, Aryan Esfandiari, C. Victor J...",CoRR,2020,AIM 2020 Challenge on Video Extreme Super-Reso...,https://arxiv.org/abs/2009.06290,art
4,Norimichi Ukita,"Kazutoshi Akita,Kyotaro Tokoro",CoRR,2024,Burst Super-Resolution with Diffusion Models f...,https://doi.org/10.48550/arXiv.2403.19428,art
...,...,...,...,...,...,...,...
26701427,Juan Gutierrez,"Ashley Houston-King, Cliff Freeman, Eli Tucke...",FabLearn/Constructionism,2023,Youth Pedagogical Development in Youth Teachin...,https://doi.org/10.1145/3615430.3615444,inp
26701428,Ashley Houston-King,"Cliff Freeman, Eli Tucker-Raymond, Katherine ...",FabLearn/Constructionism,2023,Youth Pedagogical Development in Youth Teachin...,https://doi.org/10.1145/3615430.3615444,inp
26701429,Cliff Freeman,"Ashley Houston-King, Eli Tucker-Raymond, Kath...",FabLearn/Constructionism,2023,Youth Pedagogical Development in Youth Teachin...,https://doi.org/10.1145/3615430.3615444,inp
26701430,Katherine K. Frankel,"Ashley Houston-King, Cliff Freeman, Eli Tucke...",FabLearn/Constructionism,2023,Youth Pedagogical Development in Youth Teachin...,https://doi.org/10.1145/3615430.3615444,inp


In [39]:
#seek coauthors that do not appear as egos

ego_set = set(ego_df['ego'])

coauthor_set = set()
for coauthor_str in ego_df['coauthors']:
    if pd.notna(coauthor_str) and coauthor_str != '':
        coauthor_set.update([c.strip() for c in coauthor_str.split(',')])

missing_egos = coauthor_set - ego_set

if missing_egos:
    new_rows = pd.DataFrame({
        'ego': list(missing_egos),
        'coauthors': [''] * len(missing_egos)
    })
    ego_df = pd.concat([ego_df, new_rows], ignore_index=True)

ego_df

,ego,coauthors,source,year,title,doi,Type
0,Kodai Shimosato,Norimichi Ukita,CoRR,2024,Inpainting-Driven Mask Optimization for Object...,https://doi.org/10.48550/arXiv.2403.15849,art
1,Kodai Shimosato,Norimichi Ukita,IEEE Access,2021,Multi-Modal Data Fusion for Land-Subsidence Im...,https://doi.org/10.1109/ACCESS.2021.3120133,art
2,Norimichi Ukita,Kodai Shimosato,CoRR,2024,Inpainting-Driven Mask Optimization for Object...,https://doi.org/10.48550/arXiv.2403.15849,art
3,Norimichi Ukita,"Arnau Raventos, Aryan Esfandiari, C. Victor J...",CoRR,2020,AIM 2020 Challenge on Video Extreme Super-Reso...,https://arxiv.org/abs/2009.06290,art
4,Norimichi Ukita,"Kazutoshi Akita,Kyotaro Tokoro",CoRR,2024,Burst Super-Resolution with Diffusion Models f...,https://doi.org/10.48550/arXiv.2403.19428,art
...,...,...,...,...,...,...,...
28630100,Geetanjali R. Kamath,,NaN,NaN,NaN,NaN,NaN
28630101,Shigeaki Matsuoka,,NaN,NaN,NaN,NaN,NaN
28630102,Martin Koppers,,NaN,NaN,NaN,NaN,NaN
28630103,Ehren Dohler,,NaN,NaN,NaN,NaN,NaN


In [ ]:
import re

ego_df["ego_number"]=ego_df["ego"].apply(lambda x: re.findall(r'\d+',x))
ego_df["ego_number"]=ego_df["ego_number"].str[0]
ego_df["ego"]=ego_df["ego"].apply(lambda x: re.sub(r"\d","",x))
ego_df["name"]=ego_df["ego"].str.lstrip().str.split(" ").str[0].str.lower()
ego_df[~ego_df["ego_number"].isna()]

,ego,coauthors,source,year,title,doi,Type,ego_number
65,Zheyuan Hu,"Ameya D. Jagtap, George Em Karniadakis, Kenji...",CoRR,2021,When Do Extended Physics-Informed Neural Netwo...,https://arxiv.org/abs/2109.09444,art,0002
66,Zheyuan Hu,"George Em Karniadakis, Kenji Kawaguchi, Khemr...",CoRR,2023,Tackling the Curse of Dimensionality with Phys...,https://doi.org/10.48550/arXiv.2307.12306,art,0002
67,Zheyuan Hu,"Ameya D. Jagtap, George Em Karniadakis, Kenji...",CoRR,2022,Augmented Physics-Informed Neural Networks (AP...,https://doi.org/10.48550/arXiv.2211.08939,art,0002
68,Zheyuan Hu,"Ameya D. Jagtap, George Em Karniadakis, Kenji...",SIAM J. Sci. Comput.,2022,When Do Extended Physics-Informed Neural Netwo...,https://doi.org/10.1137/21m1447039,art,0002
69,Zheyuan Hu,"George E. Karniadakis, Kenji Kawaguchi, Zhong...",SIAM J. Sci. Comput.,2025,Score-Based Physics-Informed Neural Networks f...,https://doi.org/10.1137/24m1638768,art,0002
...,...,...,...,...,...,...,...,...
28629841,Jun Cui,,NaN,NaN,NaN,NaN,NaN,0002
28629859,Jianfeng Liu,,NaN,NaN,NaN,NaN,NaN,0003
28629942,Kun Zhao,,NaN,NaN,NaN,NaN,NaN,0011
28630016,James Chapman,,NaN,NaN,NaN,NaN,NaN,0003


In [47]:

with fs.open("s3://lab/main_df.parquet", 'wb') as f:
    ego_df.to_parquet(f, index=False)

In [27]:
import numpy as np
ego_df["name"]=np.where(len(ego_df["ego"].str.split())>1, ego_df["ego"].str.split(n=1).str[0], None)
def is_valid_firstname(word):
    if not isinstance(word, str):  # skip NaN/None
        return False
    if len(word) == 1:  # single letter
        return False
    if word.isupper():  # all caps (acronyms)
        return False
    if not re.match(r'^[A-Za-z-]+$', word):  # contains only letters or hyphens
        return False
    return True

ego_df["name"] = ego_df["name"].apply(lambda x: x if is_valid_firstname(x) else None)


In [1]:
ego_df["name"]=ego_df["ego"].str.lstrip().str.split(" ").str[0].str.lower()


NameError: name 'ego_df' is not defined

In [32]:
names=names.drop_duplicates()

In [8]:
ego_df.to_parquet("ego_df.parquet",index=False)


In [19]:
with fs.open("s3://lab/name_gender_dataset.csv") as f:
    g1 = pd.read_csv(f)
with fs.open("s3://lab/wgnd_2_0_name-gender-langcode.csv") as f:
    g2 = pd.read_csv(f)

g1=g1[["Name","Gender"]]
g2[["Name","Gender"]]=g2[["name","gender"]]
g2=g2[["Name","Gender"]]

In [20]:
g2[100:150]

,Name,Gender
100,a laura,F
101,a lelia,F
102,a lelia,F
103,a lerah,F
104,a lerah,F
105,a lex ander,M
106,a lex ander,M
107,a lie,M
108,a lisa,F
109,a lisa,F


In [22]:
pd.concat([g1,g2],axis=0,ignore_index=True)

,Name,Gender
0,James,M
1,John,M
2,Robert,M
3,Michael,M
4,William,M
...,...,...
21978307,히카리,F
21978308,히토미,F
21978309,힘찬,M
21978310,凉峰,M
